In [5]:
# 初始化
import numpy as np
# 导入 Qiskit 库
from qiskit import QuantumCircuit, transpile, quantum_info
from qiskit_aer import QasmSimulator
from qiskit import ClassicalRegister, QuantumRegister
from qiskit.visualization import plot_histogram

# 设置量子比特数量
n = 3

# 定义 Oracle 函数，用于标记特定的量子态 |111>
def Oracle():  
    oc = QuantumCircuit(n)  # 创建一个量子电路，包含n个量子比特
    # 使用H门和CCX门（受控非门）来实现 CZ 门的效果
    oc.h(2)  # 对第2个量子比特应用Hadamard门
    oc.ccx(0, 1, 2)  # 应用受控非门
    oc.h(2)  # 再次对第2个量子比特应用Hadamard门
    return oc

# 定义扩散算子函数，用于实现幅度放大
def A(nb): 
    ac = QuantumCircuit(nb)  # 创建一个量子电路，包含nb个量子比特
    ac.h(range(nb))  # 对所有量子比特应用Hadamard门
    ac.x(range(nb))  # 对所有量子比特应用X门（非门）
    # 执行多控制Z门
    ac.h(nb-1)  # 对最后一个量子比特应用Hadamard门
    ac.mcx(list(range(nb-1)), nb-1)  # 多控制Z门
    ac.h(nb-1)  # 再次对最后一个量子比特应用Hadamard门
    ac.x(range(nb))  # 对所有量子比特应用X门
    ac.h(range(nb))  # 对所有量子比特再次应用Hadamard门
    return ac

# 创建量子电路
qc = QuantumCircuit(n)
qc.h(range(n))  # 对所有量子比特应用Hadamard门

# 构建 Oracle 和幅度放大电路
OA = QuantumCircuit(n)
# 查询 Oracle
OA.compose(Oracle(), inplace=True)  # 将Oracle函数内的内容组合到OA电路中
# 幅度放大
OA.compose(A(n), inplace=True)  # 将扩散算子组合到OA电路中

# 将OA电路放在H门之后
qc.compose(OA, inplace=True)  # 将OA电路组合到qc电路中

# 绘制电路图
qc.draw(output='mpl',filename='grover_circuit.png')

# 不要调用 measure()，因为我们想在状态向量中看到概率幅度
qc.save_statevector()  # 保存量子电路的状态向量

# 使用 Aer 的 qasm_simulator
simulator = QasmSimulator()  # 创建一个模拟器实例

# 将电路编译为低级别的 QASM 指令
compiled_circuit = transpile(qc, simulator)  # 编译量子电路

# 在 qasm 模拟器上执行电路
job = simulator.run(compiled_circuit, shots=1000)  # 运行量子电路1000次

# 从任务中获取结果
result = job.result()  # 获取运行结果
out_state = result.get_statevector()  # 获取状态向量
counts = result.get_counts()  # 获取测量结果的计数

# 我们必须反转字典 counts 中的字符串
answer = {}
for str in counts:
    answer[str[::-1]] = counts[str]  # 反转字符串并存储到新字典answer中

# 绘制直方图
plot_histogram(answer).savefig('groverout.png')  # 显示结果的直方图